In [1]:
import pandas as pd
import numpy as np
from HelperFunctions import subset_df_and_rename_columns, merge_two_df
Lig4_df = pd.read_csv('/Users/tianji/Desktop/Alu Project/DSE_Results_Data/LIG4KD/fisher_exact_test_results/Lig4.HCT116.Cass.CtrlvsKD.AlterSpliceResults.txt', sep = '\t')
MAVS_df = pd.read_csv('/Users/tianji/Desktop/Alu Project/DSE_Results_Data/MAVSKO/fisher_exact_test_results/MAVSKO.HCT116.Cass.CtrlvsKuKD.AlterSpliceResults.txt', sep = '\t')
print(f"Lig4: {len(Lig4_df)}")
print(f"MAVS: {len(MAVS_df)}")

# Rename the I_g1 Ctrl column for each data frame 
Lig4_df = Lig4_df.rename(columns = {'I_g1(Lig4Ctrl)':'I_g1(Lig4_Ctrl)', 'I_g2(Lig4KD)':'I_g2(Lig4_KD)'})
MAVS_df = MAVS_df.rename(columns = {'I_g1(Ctrl(MAVSKO))':'I_g1(MARVSKO_Ctrl)', 'I_g2(KuKD(MAVSKO))':'I_g2(MARVSKO_KuKD)'})
Lig4_df = subset_df_and_rename_columns(Lig4_df, 'Lig4')
MAVS_df = subset_df_and_rename_columns(MAVS_df, 'MAVSKO')
merged_df = merge_two_df(Lig4_df, MAVS_df, df2merge_suffix = 'MAVSKO', df2merge_Ig1_suffix = 'MARVSKO_Ctrl', df2merge_Ig2_suffix = 'MARVSKO_KuKD')
print(len(merged_df))
print(merged_df.columns)

main_cass_df = pd.read_csv('/Users/tianji/Desktop/Alu Project/Output/Merged_DSE_Results/Cass.AluOverlap.110325.csv')
main_cass_df = main_cass_df.iloc[:,3:]

# Extract columns after 'coverage(MAVSKO)', plus the 'name' column for merging
cols_to_merge = merged_df[['name'] + list(merged_df.columns[10:])]
result = pd.merge(main_cass_df, cols_to_merge, on='name', how='left')

/Users/tianji/mambaforge/envs/myenv/lib/python3.12/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


Lig4: 42761
MAVS: 42761
42761
Index(['gene', 'chrom', 'chromStart', 'chromEnd', 'chromLength', 'name',
       'score', 'strand', 'type', 'isoformIDs', 'coverage(Lig4)',
       'I_g1(Lig4_Ctrl)', 'I_g2(Lig4_KD)', 'dI_g1_vs_g2(Lig4)', 'pvalue(Lig4)',
       'FDR(Lig4)', 'filter(Lig4)', 'coverage(MAVSKO)', 'I_g1(MARVSKO_Ctrl)',
       'I_g2(MARVSKO_KuKD)', 'dI_g1_vs_g2(MAVSKO)', 'pvalue(MAVSKO)',
       'FDR(MAVSKO)', 'filter(MAVSKO)'],
      dtype='object')


/var/folders/gp/l_62hzx91kz0hng_60kps5t00000gn/T/ipykernel_3854/2733477276.py:18: DtypeWarning: Columns (234) have mixed types. Specify dtype option on import or set low_memory=False.
  main_cass_df = pd.read_csv('/Users/tianji/Desktop/Alu Project/Output/Merged_DSE_Results/Cass.AluOverlap.110325.csv')


In [2]:
# Add a new column 'alu_for_count' to categorize CE into exonic asAlu, intronic-inter, intronic-intra, intronic-other, no Alu, and exonic ssAlu

# Create a new alu type column that specifies 'other without Alu' and 'other with Alu'
result['alu_type_for_count'] = result['alu_type']

# Put "other_without_Alu" into its own bucket as requested
result.loc[
    result['alu_subtype'] == 'other_without_Alu',
    'alu_type_for_count'
] = 'no Alu'

result.loc[
    ((result['alu_specific_subtype'] == 'other_with_irAlu_only_ui') | (result['alu_specific_subtype'] == 'other_with_irAlu_only_di')),
    'alu_type_for_count'
] = 'irAlu_intra'


result['alu_type_for_count'] = result['alu_type_for_count'].replace({'irAlu': 'irAlu_inter', 'other': 'irAlu_other'})

result['alu_type_for_count'].value_counts()


alu_type_for_count
irAlu_inter    14941
no Alu          9859
irAlu_other     8000
irAlu_intra     6859
asAlu           2678
ssAlu            424
Name: count, dtype: int64

In [ ]:
# SAVE
#result.to_csv('/Users/tianji/Desktop/Alu Project/Output/Merged_DSE_Results/Cass.AluOverlap.Ctrl.110325.csv')